## Import

In [ ]:
import pandas as pd
import numpy as np
import os


from itertools import takewhile, dropwhile
from collections import Counter

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dataframes_util as ut

# Import base df

In [ ]:
if (not os.path.isfile('../tweets.csv')):
    %run userPreparation.ipynb

dfPosts = pd.read_csv('../tweets.csv',low_memory=False)

In [ ]:
dfPosts

## Filtered by retweet

### High credibility

In [ ]:
listHighCred =  ['tagadala7', 'La7tv', 'Corriere', 'tg2rai', 'TgrRaiPuglia', 'Linkiesta', 'ilfoglio_it', 
                 'fanpage', 'LaStampa', 'RaiNews', 'fattoquotidiano', 'TgrRaiVeneto', 'agorarai', 'TgrRaiSicilia', 
                 'UnioneSarda', 'SkyTG24', 'repubblica', 'TgrRai', 'Agenzia_Ansa', 'Tg3web', 'TgrRaiToscana', 
                 'Affaritaliani', 'TgrRaiTrentino', 'PiazzapulitaLA7', 'Adnkronos', 'agerpres', 'RepubblicaTv', 
                 'TgrRaiFVG', 'ilpost', 'Open_gol', 'Radio1Rai', 'MediasetTgcom24', 'TgLa7', 'RaiStudio24', 
                 'TgrRaiMolise', 'sole24ore', 'Ticinonline', 'ricpuglisi', 'SimoneCosimi', 'giusmo1', 'RobertoBurioni',
                 'rtl1025', 'messveneto', 'HuffPostItalia', 'Agenzia_Italia', 'ItaliaViva', 'iltirreno', 'Agenzia_Dire',
                 'stebellentani', 'eziomauro', 'QRepubblica', 'lucianocapone', 'robersperanza', 'vocedelpatriota', 
                 'GiovaQuez', 'Cartabellotta', 'istsupsan', 'christianrocca', 'Zeta_Luiss', 'radioanchio', '24Mattino', 
                 'sottoinchiesta', 'riotta']

#### Retweeted by an high credibility user

#### Retweet  an high credibility user

In [ ]:
#Account that have retweeted a suspect (follower)
userHighCred = listHighCred
dfRetweet = pd.DataFrame()
print ("Generation ",0, ": ",len(userHighCred))

for i in range(0, 1):
    dfRetweet = ut.retweet_a_suspect(dfPosts,userHighCred)
    userHighCred = userHighCred + list(dfRetweet[dfRetweet['rt_rate']>=10.0].index)
    userHighCred = list(set(userHighCred))
    print ("Generation ",i+1, ": ",len(userHighCred))
dfRetweet

In [ ]:
fig = px.histogram(x=dfRetweet[dfRetweet['rt_rate']>0]["rt_rate"],
                 histnorm='percent',title="Retweet rate on follower (high credibility)")
fig.update_xaxes(title='retweet rate')
fig.show()

In [ ]:
dfProvax = dfPosts[dfPosts.isin(userHighCred)['user_screen_name']].copy()

### NoVax

In [ ]:
listSuspect = ['IacobellisT','Piero42395724', 'TommyBrain','xenonian1', 'Z3r0Rules','Pietro_Otto',
               'MarySpes','manu_etoile','CarpaneseSilva1','il_brigante07','Sakurauchi_Hime','MinervaMcGrani1',
              'marchesaangeli','lucabattanta',
              'gael99','LPinicia','Mariang47614228','SoniaLaVera','valy_s','Samira1577','daniele19921','d_essere'
              'anto_galli4','LuigiF97101292','EureosCriss','vaniacavi',
              'markred17','RenzoCianchetti','12qbert','ManuQ24916888','bisagnino','thewaterflea',
              'BarbaraRaval','noitre32','intuslegens','pbecchi','miia_2018','ladyonorato','cris_cersei','RadioSavana',
              'lameduck1960','a_meluzzi','francescatotolo','Mr_Ozymandias','FmMosca','LaVeritaWeb','Bluefidel47',
              'fdragoni','byoblu','MinutemanItaly','TarroGiulio','NicolaPorro','25O319','sabrina__sf','liliaragnar',
              'FabioFranchi1','EliseiNicole']

#### Retweeted by a suspect
Any user that is defined as a novax that retweets a user, the retweeted user will automatically be considered a novax

#### Retweet a suspect

In [ ]:
#Account that have retweeted a suspect (follower)
userNovax = listSuspect
dfRetweet = pd.DataFrame()
print ("Generation ",0, ": ",len(userNovax))
for i in range(0, 2):
    dfRetweet = ut.retweet_a_suspect(dfPosts,userNovax)
    userNovax = userNovax + list(dfRetweet[dfRetweet['rt_rate']>=20.0].index)
    userNovax = list(set(userNovax))
    
    print ("Generation ",i+1, ": ",len(userNovax))

dfRetweet

In [ ]:
fig = px.histogram(x=dfRetweet[dfRetweet['rt_rate']>0]["rt_rate"],
                 histnorm='percent',title="Retweet rate on follower (novax)")
fig.update_xaxes(title='retweet rate')
fig.show()

In [ ]:
dfNovax = dfPosts[dfPosts.isin(userNovax)['user_screen_name']].copy()

# .CSV Creation 

### Novax

In [ ]:
dfNovax.drop_duplicates(['id'],inplace=True)
dfNovax

### High credibility

In [ ]:
dfProvax.drop_duplicates(['id'],inplace=True)
dfProvax

### Other user

In [ ]:
idNovax = dfNovax.groupby('id').first().index
dfOtherElem = dfPosts[dfPosts['id'].isin(idNovax) == False]

idProvax = dfProvax.groupby('id').first().index
dfOtherElem = dfOtherElem[dfOtherElem['id'].isin(idProvax) == False]

dfOtherElem

### Write dataframe to .csv

In [ ]:
dfProvax.to_csv('retweet_class/tweets_provax.csv', index=False)

In [ ]:
dfNovax.to_csv('retweet_class/tweets_novax.csv', index=False)

In [ ]:
dfOtherElem.to_csv('retweet_class/tweets_otherUser.csv', index=False)

In [ ]:
#Extract to another dataframe info about doubled classified users
doubled_class = list(np.array(dfNovax['user_id'])[pd.Series(dfNovax['user_id']).isin(dfProvax['user_id'])])
dfDoubled = dfPosts[dfPosts['user_id'].isin(doubled_class)]

dfDoubled.to_csv('retweet_class/tweets_doubled_classified.csv', index=False)